### LangChain Agents and Tools

In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.utilities import BingSearchAPIWrapper

In [2]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
temperature = 1.0
max_tokens = 512
model_name = "gpt-3.5-turbo-0125"

llm = ChatOpenAI(
    model=model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key=openai_api_key
)

In [3]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """
        Multiplies two integers
    """

    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    """
        Adds or Sums two integers
    """

    return first_int + second_int


@tool
def exponentize(base: int, exponent: int) -> int:
    """
        Exponentize the base to the exponent value
    """

    return base ** exponent


@tool("BingSearch")
def search(search_query: str) -> str:
    """
        Useful to search for any kinds of information.
        Useful when you need to search the internet for any kinds of information.
    """

    search_wrapper = BingSearchAPIWrapper()

    return search_wrapper.run(search_query)

In [ ]:
tools = [add, multiply, exponentize, search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [ ]:
question = """
    Take 3 to the fifth power and multiply that by the sum of twelve and three, and 
    then square the result.
    at last Subtract the result by 10.
    What's the capital of India? 
"""

response = agent_executor.invoke({
    "input": question
})

print(response)